In [1]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Function to parse XML annotation for bounding box
def parse_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    boxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        boxes.append((int(bbox.find('xmin').text),
                      int(bbox.find('ymin').text),
                      int(bbox.find('xmax').text),
                      int(bbox.find('ymax').text)))
    return boxes

# Function to load and crop images based on bounding box
def load_and_crop_image(image_path, box):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Ensure box dimensions are positive
    box_width = box[2] - box[0]
    box_height = box[3] - box[1]
    if box_width <= 0 or box_height <= 0:
        print(f"Invalid box dimensions (width: {box_width}, height: {box_height}) for image: {image_path}")
        return None

    image = tf.image.crop_to_bounding_box(image, box[1], box[0], box_height, box_width)
    image = tf.image.resize(image, (224, 224))  # Resize for model compatibility
    return image.numpy()  # Convert to numpy array for model compatibility


# Function to prepare dataset for 'person' class
def prepare_person_dataset(base_path, annotation_folder, image_folders):
    images, labels = [], []

    annotations_path = os.path.join(base_path, annotation_folder)
    for annotation_file in os.listdir(annotations_path):
        filename = annotation_file.replace('.xml', '.jpg')
        annotation_path = os.path.join(annotations_path, annotation_file)
        boxes = parse_annotation(annotation_path)

        for folder in image_folders:
            image_path = os.path.join(base_path, folder, filename)
            if os.path.exists(image_path):
                for box in boxes:
                    cropped_image = load_and_crop_image(image_path, box)
                    if cropped_image is not None:
                        images.append(cropped_image)
                        labels.append(1)  # Label for 'person'
                break

    return np.array(images), np.array(labels)


# Function to prepare dataset for 'non-person' class
def prepare_non_person_dataset(base_path, list_paths, image_folder):
    images, labels = [], []

    for list_path in list_paths:
        with open(os.path.join(base_path, list_path), 'r') as file:
            lines = file.readlines()
            for line in lines:
                image_name, _, _, _ = line.split()
                image_path = os.path.join(base_path, image_folder, image_name + '.jpg')
                try:
                    image_raw = tf.io.read_file(image_path)  # Read the raw image file
                    image = tf.image.decode_jpeg(image_raw, channels=3)  # Decode JPEG image
                    image = tf.image.resize(image, (224, 224))  # Resize for model compatibility
                    images.append(image.numpy())  # Convert to numpy array for model compatibility
                    labels.append(0)  # Label for 'non-person'
                except tf.errors.NotFoundError:
                    print(f"The file at {image_path} does not exist.")
                except Exception as e:
                    print(f"Error processing file {image_path}: {e}")

    return np.array(images), np.array(labels)

# Example usage
with tf.device('/cpu:0'):
    base_path_person = 'raw_data/llvip/raw_data'
    image_folders_person = ['infrared/train', 'infrared/test']
    annotation_folder_person = 'Annotations'
    person_images, person_labels = prepare_person_dataset(base_path_person, annotation_folder_person, image_folders_person)
    
    base_path_non_person = 'raw_data/oxford-iiit-pet/raw_data'
    list_paths_non_person = ['annotations/annotations/trainval.txt', 'annotations/annotations/test.txt']
    image_folder_non_person = 'images/grayscale'
    non_person_images, non_person_labels = prepare_non_person_dataset(base_path_non_person, list_paths_non_person, image_folder_non_person)
    
    # Combine datasets
    images = np.concatenate((person_images, non_person_images))
    labels = np.concatenate((person_labels, non_person_labels))
    
    # Split combined dataset into training, test, and then training into training and validation sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # Convert labels to categorical for model training
    y_train = to_categorical(y_train, num_classes=2)
    y_val = to_categorical(y_val, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

# At this point, X_train, X_val, X_test, y_train, y_val, and y_test are ready for training, validating, and testing your model.


2024-03-26 12:06:16.128117: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-26 12:06:16.128137: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2024-03-26 12:06:16.128141: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2024-03-26 12:06:16.128154: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-26 12:06:16.128164: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Invalid box dimensions (width: 0, height: 7) for image: raw_data/llvip/raw_data/infrared/train/090437.jpg
Invalid box dimensions (width: 0, height: 1) for image: raw_data/llvip/raw_data/infrared/train/020118.jpg
Invalid box dimensions (width: 0, height: 1) for image: raw_data/llvip/raw_data/infrared/train/020236.jpg
Invalid box dimensions (width: 0, height: 7) for image: raw_data/llvip/raw_data/infrared/train/170350.jpg
Invalid box dimensions (width: 0, height: 1) for image: raw_data/llvip/raw_data/infrared/train/091180.jpg


2024-03-26 12:10:47.743198: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_14.jpg; No such file or directory
2024-03-26 12:10:47.743452: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: NOT_FOUND: raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_14.jpg; No such file or directory
2024-03-26 12:10:47.763758: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_156.jpg; No such file or directory
2024-03-26 12:10:47.763769: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: NOT_FOUND: raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_156.jpg; No such file or directory
2024-03-26 12:10:47.837353: W tensorflow/core/framework/op_kerne

The file at raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_14.jpg does not exist.
The file at raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_156.jpg does not exist.
The file at raw_data/oxford-iiit-pet/raw_data/images/grayscale/Egyptian_Mau_186.jpg does not exist.


2024-03-26 12:10:49.322162: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: raw_data/oxford-iiit-pet/raw_data/images/grayscale/Abyssinian_5.jpg; No such file or directory
2024-03-26 12:10:49.322175: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: NOT_FOUND: raw_data/oxford-iiit-pet/raw_data/images/grayscale/Abyssinian_5.jpg; No such file or directory


The file at raw_data/oxford-iiit-pet/raw_data/images/grayscale/Abyssinian_5.jpg does not exist.


In [2]:
print(len(X_train))
print(len(y_train))

31856
31856


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPooling2D, Flatten, Dense, Convolution2D, Activation, Input, BatchNormalization
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')

model = Sequential()

model.add(Input(shape=(224, 224, 3)))
model.add(Convolution2D(32,(3,3),padding='same'))
model.add(BatchNormalization())
#model.add(Convolution2D(32,(3,3),padding='same'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))

#model.add(Convolution2D(32,(3,3),padding='same'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))

#model.add(Convolution2D(32,(3,3),padding='same'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(40))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('sigmoid'))

# compile model and initialize weights
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 401408)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │    16,056,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            82 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,057,466 (61.25 MB)

 Trainable params: 16,057,402 (61.25 MB)

 Non-trainable params: 64 (256.00 B)

In [ ]:
with tf.device('/cpu:0'):
    history = model.fit(X_train, y_train,
        batch_size=32,  # Adjust based on your dataset size and memory constraints
        epochs=5,  # Adjust based on the desired number of training epochs
        validation_data=(X_val, y_val)) 

Epoch 1/5


2024-03-26 13:18:24.060542: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


138/996 ━━━━━━━━━━━━━━━━━━━━ 6:23 448ms/step - accuracy: 0.9262 - loss: 7.7205

In [ ]:
print(history.history)

# Corrected plotting code
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)  # Corrected subplot call
plt.plot(history.history['accuracy'], linestyle='-.')
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)  # Corrected subplot call
plt.plot(history.history['loss'], linestyle='-.')
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.show()